<a href="https://colab.research.google.com/github/agupte87/LearningPyspark/blob/master/Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd ..
%mkdir spark

/


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [3]:
!wget -P /spark/ https://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz

--2021-07-21 14:29:54--  https://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz
Resolving apache.osuosl.org (apache.osuosl.org)... 64.50.233.100, 140.211.166.134, 64.50.236.52, ...
Connecting to apache.osuosl.org (apache.osuosl.org)|64.50.233.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224762914 (214M) [application/x-gzip]
Saving to: ‘/spark/spark-3.0.3-bin-hadoop3.2.tgz’

spark-3.0.3-bin-had 100%[===================>] 214.35M  30.4MB/s    in 13s     

2021-07-21 14:30:07 (16.8 MB/s) - ‘/spark/spark-3.0.3-bin-hadoop3.2.tgz’ saved [224762914/224762914]



In [4]:
!tar xf /spark/spark-3.0.3-bin-hadoop3.2.tgz 
%mv spark-3.0.3-bin-hadoop3.2 /spark/




In [5]:
!pip install -q findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/spark/spark-3.0.3-bin-hadoop3.2"

In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [8]:
%cd /content/
%mkdir /chapterData 
%cd /chapterData
%mkdir retail-data
%cd retail-data/
%mkdir by-day
%cd by-day

/content
/chapterData
/chapterData/retail-data
/


In [9]:
%pwd


'/'

In [10]:

!wget -P  'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-01.csv'
!wget -P  'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-02.csv'
!wget -P  'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-03.csv'
!wget -P  'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-04.csv'
!wget -P  'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-05.csv'
!wget -P  'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-06.csv'
!wget -P  'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-07.csv'


--2021-07-21 14:30:20--  https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-01.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275001 (269K) [text/plain]
Saving to: ‘content/chapterData/retail-data/by-day/2010-12-01.csv’

2010-12-01.csv      100%[===================>] 268.56K  --.-KB/s    in 0.007s  

2021-07-21 14:30:20 (37.0 MB/s) - ‘content/chapterData/retail-data/by-day/2010-12-01.csv’ saved [275001/275001]

--2021-07-21 14:30:20--  https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-02.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githu

# New Section

In [11]:
staticDF = spark.read.format("csv")\
.option("inferSchema", "true")\
.option("header", "true")\
.load("/content/chapterData/retail-data/by-day/*.csv")

In [12]:
staticDF.show(10)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   537226|    22811|SET OF 6 T-LIGHTS...|       6|2010-12-06 08:34:00|     2.95|   15987.0|United Kingdom|
|   537226|    21713|CITRONELLA CANDLE...|       8|2010-12-06 08:34:00|      2.1|   15987.0|United Kingdom|
|   537226|    22927|GREEN GIANT GARDE...|       2|2010-12-06 08:34:00|     5.95|   15987.0|United Kingdom|
|   537226|    20802|SMALL GLASS SUNDA...|       6|2010-12-06 08:34:00|     1.65|   15987.0|United Kingdom|
|   537226|    22052|VINTAGE CARAVAN G...|      25|2010-12-06 08:34:00|     0.42|   15987.0|United Kingdom|
|   537226|    22705|   WRAP GREEN PEARS |      25|2010-12-06 08:34:00|     0.42|   15987.0|United Kingdom|
|   537226|    20781|GOLD EA

In [13]:
dayOfLargePurchase = staticDF.groupby("InvoiceDate").sum("UnitPrice").withColumnRenamed("sum(UnitPrice)", "SumOFUnitPrice")
dayOfLargePurchase.sort(dayOfLargePurchase.InvoiceDate.desc()).take(10)

[Row(InvoiceDate='2010-12-07 18:36:00', SumOFUnitPrice=3118.1100000000033),
 Row(InvoiceDate='2010-12-07 17:19:00', SumOFUnitPrice=39.16),
 Row(InvoiceDate='2010-12-07 17:05:00', SumOFUnitPrice=1.95),
 Row(InvoiceDate='2010-12-07 16:43:00', SumOFUnitPrice=128.26999999999998),
 Row(InvoiceDate='2010-12-07 16:42:00', SumOFUnitPrice=35.62),
 Row(InvoiceDate='2010-12-07 16:21:00', SumOFUnitPrice=10.95),
 Row(InvoiceDate='2010-12-07 16:20:00', SumOFUnitPrice=24.599999999999998),
 Row(InvoiceDate='2010-12-07 15:59:00', SumOFUnitPrice=63.300000000000004),
 Row(InvoiceDate='2010-12-07 15:54:00', SumOFUnitPrice=153.59),
 Row(InvoiceDate='2010-12-07 15:51:00', SumOFUnitPrice=6706.71)]

In [14]:
from pyspark.sql.functions import window , column , desc , col 
staticDataFrame = staticDF.selectExpr("CustomerID", "(UnitPrice*Quantity) as total_purchase_value", "InvoiceDate").groupBy(\
    col("CustomerId"),\
    window(col("InvoiceDate"),"1 day"))\
.sum("total_purchase_value")\
.withColumnRenamed("sum(total_purchase_value)", "total_purchase_value")\
.show(10)



+----------+--------------------+--------------------+
|CustomerId|              window|total_purchase_value|
+----------+--------------------+--------------------+
|   13408.0|[2010-12-01 00:00...|  1024.6800000000003|
|   17460.0|[2010-12-01 00:00...|                19.9|
|   16950.0|[2010-12-07 00:00...|               172.0|
|   13269.0|[2010-12-05 00:00...|              351.43|
|   12647.0|[2010-12-05 00:00...|               372.0|
|   15235.0|[2010-12-01 00:00...|                79.5|
|   13495.0|[2010-12-06 00:00...|  510.94999999999993|
|   14506.0|[2010-12-05 00:00...|  269.69999999999993|
|   14911.0|[2010-12-06 00:00...|  1182.5000000000002|
|   13295.0|[2010-12-02 00:00...|               -3.25|
+----------+--------------------+--------------------+
only showing top 10 rows

